In [ ]:
## Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Output import Output
from IO.Input import Input
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

from pyspark.ml.classification import NaiveBayes

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel


import math
import time


def writeCoef(path,snpsIds,sc,idToName,corr, name = None):
        
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt "  
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
        
    snps = []
    for i in range(len(snpsIds)):
        s = snpsIds[i]
        #snps.append(idToName[s])
        snps.append(s)
            
    print("snpsIds = ",len(snpsIds))
    print("idToName = ",len(idToName))
        
    write = open(p,'w')
    write.write("len = "+str(len(sc))+'\n')
    write.write('corr = '+str(corr)+'\n')
    for i in range(len(snps)):
            
        write.write(str(snps[i])+'\t'+str(sc[i])+'\n')
            
    write.close()


def calcCoeff(path,coefs, columns,corr):
    
    ids = {}
    ids['coef']={}
   
    print("before = ",len(set(coefs)))
    #for i in range(len(coefs)):
     #   coefs[i] = abs(coefs[i])

    print("after = ",len(set(coefs)))

    idToName = {}
    nameToId = {}

    for i in range(len(coefs)):
        nameToId[coefs[i]] = []

    for i in range(len(coefs)):
        nameToId[coefs[i]].append(i)
        idToName[i] = coefs[i]


    ids['coef']['nameToId'] = nameToId
    ids['coef']['idToName'] = idToName


    sc = sorted(coefs,reverse=True)

    top_30 = []

    for i in range(30):

        snp = ids['coef']['nameToId'][sc[i]][0]
        ids['coef']['nameToId'][sc[i]].remove(snp)
        top_30.append(columns[snp])
     #   top_30.append(snp)

    #snpReduc['low'] = top_30
    writeCoef(path,top_30,sc,columns,corr, name = 'mycutoffabs')
    
    
def showMetrics(c,value=1):
    
    predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
    metrics = MulticlassMetrics(predictionAndLabels)

    print("confusion matrix = ", metrics.confusionMatrix().toArray())

    print("accuracy = ", metrics.accuracy)
    print("recal = ", metrics.recall(value))
    print("precision = ", metrics.precision(value))
    print("f1 = ", metrics.fMeasure(1.0))
    
    
def showMyMetric(results,v1 = 1,v2 = 0):
    
    if v1 == v2:
        print("wrong values!!!v1 is the same with v2!!!!")
       
    
    r = results.rdd
    r3 = r.collect()

    same0 = 0
    same1 = 0
    sum0 = 0
    sum1 = 0
    sumall = 0
    for i in r3:

        if i[v1] == 0:
            sum0 += 1
            if i[v2] == 0:
                same0 += 1
        elif i[v1] == 1:
            sum1 += 1
            if i[v2] == 1:
                same1 += 1
        sumall += 1

    print('sum0 = ', sum0)
    print('sum1 = ', sum1)
    print('same0 = ', same0)
    print('same1 = ', same1)
    print('all = ', sumall)
    print('all2 = ', sum0+sum1)

In [ ]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
    .set('spark_executor_cores',"3")
    .set('spark.graphx.pregel.checkpointInterval','-1')
    .set('spark.network.timeout','100000000')
    .set('spark.executor.heartbeatInterval','10000000'))

#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [ ]:
#spark.stop()

# Tests

In [ ]:
path = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/'

#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snpCodeTest.csv'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
#pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snpCodeTest1.csv'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'


data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [ ]:
print("data columns = ",len(data.columns))

features = []
for i in data.columns:
    if 'rs' not in i and i !='label':
        features.append(i)
print(len(features))
#print((features))

In [ ]:
#data.select('patients','label').show(10)

In [ ]:
from pyspark.ml.linalg import DenseVector

d = data.drop('patients')
for i in features:
    d = d.drop(i)
print("columns = ",len(d.columns))

train_data,test_data = d.randomSplit([0.8,0.2],seed=18)



In [ ]:
'''t0 = train_data.count()
t1 = test_data.count()

print("test = ",t1)
print("train = ",t0)'''

# Balanced Label1 to train

In [ ]:
label1 = train_data.filter("label == 1").count()
label0 = train_data.filter("label == 0").count()
print("label0 = ", label0)
print("label1 = ", label1)


l0 = label1 * 2

x = (l0 * 100) / label0


x = x / 100

print("label1 = ", label1)
print("l2 = ",l0)
print("label0 = ", label0)
print("x = ", x)
print()

t0 = train_data.filter('label == 0')
t1 = train_data.filter('label == 1')

train0,t2 = t0.randomSplit([x,1-x],seed = 11)
train_data = train0.union(t1)

#train1,t3 = t1.randomSplit([0.5,0.5],seed = 11)
#train_data = train0.union(train1)

#t2 = t2.union(t3)
#test_data = test_data.union(t2)

t0 = train_data.count()
t1 = test_data.count()

print("label0 = ",label0)
print('label1 = ', label1)
print("l1 = ",l0)
print('len test label1 = ',train_data.filter('label==1').count())
print('len test label0 = ',train_data.filter('label==0').count())
print('train = ', t0)
print('test = ', t1)
print('all = ', t0 + t1)
print('d = ',d.count())


# UnBalanced Label1 to train

In [ ]:
label1 = train_data.filter("label == 1").count()
label0 = train_data.filter("label == 0").count()


l1 = (label1/2) + ((label1/2) * 0.5)

x = (l1 * 100) / label0


x = x / 100

print("label1 = ", label1)
print("l1 = ",l1)
print("label0 = ", label0)
print("x = ", x)

t0 = train_data.filter('label == 0')

train0,t2 = t0.randomSplit([x,1-x],seed = 11)
train1 = train_data.filter("label == 1")

train_data = train0.union(train1)
test_data = test_data.union(t2)

t0 = train_data.count()
t1 = test_data.count()

print("label0 = ",label0)
print('label1 = ', label1)
print("l1 = ",l1)
print('train = ', t0)
print('test = ', t1)
print('all = ', t0 + t1)
print('d = ',d.count())


# dokimh 

In [ ]:
from pyspark.ml.stat import Correlation

li = len(d.columns)

input_data = train_data.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
train0 = spark.createDataFrame(input_data, ["label", "features"])

final_data1 = train0.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()

In [ ]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)#oso megalytero threshold toso perissotero omoia einai 

In [ ]:

features = []

for i in range(0,len(d.columns)):
    if 'rs' in d.columns[i] and d.columns[i] !='label':
        features.append(d.columns[i])

print("len features1 = ",len(features))

snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

snpsRed1.append('label')

print('len snpsRed1 = ', len(snpsRed1))
    
features = []
for i in range(0,len(d.columns)):
    if d.columns[i] not in snpsRed1:
        features.append(d.columns[i])
print("len features2 = ",len(features))

dok_train = train_data.drop(*features)
dok_test = test_data.drop(*features)

print("columns dok train = ",len(dok_train.columns))
print("columns dok test = ",len(dok_test.columns))
li = len(dok_train.columns)

print('first column = ', dok_train.columns[0])
print('last column = ', dok_train.columns[li - 1])

print('test first column = ', dok_test.columns[0])
print('test last column = ', dok_test.columns[li - 1])

In [ ]:
li = len(dok_train.columns)
#li = len(test_data.columns)


input_data = dok_test.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
test = spark.createDataFrame(input_data, ["label", "features"])

input_data = dok_train.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
train = spark.createDataFrame(input_data, ["label", "features"])

'''input_data = test_data.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
test = spark.createDataFrame(input_data, ["label", "features"])

input_data = train_data.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))
train = spark.createDataFrame(input_data, ["label", "features"])'''

# telos dokimhs

In [ ]:
print("train = ",train.head()[1].toArray().shape)
print("test = ",test.head()[1].toArray().shape)

# Linear Regression

In [ ]:
log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train)

In [ ]:

results = lr_model.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
print("auc = ",AUC)
'''0.7909133853963526'''

In [ ]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error metricName='mae' '''

'''pipeline = Pipeline(stages=[log_reg])
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label',metricName='mse')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train)
cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)'''

In [ ]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error metricName='mae' '''

pipeline = Pipeline(stages=[log_reg])
modelEvaluator=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',metricName='accuracy')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)


cvModel = crossval.fit(train)



cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
avgResults = cvModel.avgMetrics

In [ ]:
#print(len(d.columns))
#print(d.columns[len(d.columns)-1])
calcCoeff(path,lr_model.coefficientMatrix.toArray()[0],d.columns,0.2)

In [ ]:
'''trainingSummary = cvModel.bestModel.stages[-1].summary
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))'''

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

# RFR

In [ ]:
rf = RandomForestClassifier(featuresCol='features',labelCol='label', numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=50, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)
rf_model = rf.fit(train)

In [ ]:
results = rf_model.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:

modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train)
cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

# SVM 

In [ ]:
#lsvc = LinearSVC(maxIter=10, regParam=0.1)
lsvc = LinearSVC(featuresCol='features',labelCol='label',maxIter=10)
lsvcModel = lsvc.fit(train)
lsvc.setThreshold(0.5)

In [ ]:
results = lsvcModel.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',metricName='accuracy')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=lsvc,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)


cvModel = crossval.fit(train)



cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)
lsvc.clearThreshold()

In [ ]:
#calcCoeff(path,lr_model.coefficientMatrix.toArray()[0],d.columns)

In [ ]:
'''print('len test label1 = ',train_data.filter('label==1').count())
print('len test label0 = ',train_data.filter('label==0').count())'''

# Bernoulli

In [ ]:
nb = NaiveBayes(predictionCol='prediction', labelCol='label',smoothing=0.2)
nbg = nb.fit(train)

In [ ]:
results = nbg.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',metricName='accuracy')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)


cvModel = crossval.fit(train)



cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)

# Tree

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.classification import DecisionTreeClassifier

'''features = []

for i in dok_train.columns:
    features.append(i)

assempler = VectorAssembler(inputCols=features,outputCol='features')
output = assempler.transform(data)

tr = output.select('label','features').rdd.map(lambda row: LabeledPoint(row.label, row.features))





treeD = DecisionTree.trainClassifier(tr, numClasses=2, categoricalFeaturesInfo={}, 
                                     impurity='gini', maxDepth=4, maxBins=32)'''

treeD = DecisionTreeClassifier(predictionCol='prediction', labelCol='label')
tree = treeD.fit(train)

#tree = treeD.fit(train0)
print("ok")

In [ ]:
results = tree.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',metricName='accuracy')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=treeD,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)


cvModel = crossval.fit(train)



cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)


In [ ]:
modelFile = path+'treeRules.txt'
f = open(modelFile,"w") 
f.write(tree.toDebugString)
f.close()

# Gradient-Boosted Trees (GBTs)

In [ ]:
modelGBT = GradientBoostedTrees.trainClassifier(train,categoricalFeaturesInfo={}, numIterations=10)

#modelGBT = GradientBoostedTrees(predictionCol='prediction', labelCol='label')
gbt = treeD.fit(train)

In [ ]:
results = gbt.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train)
cross_results = cvModel.transform(test)
results = cross_results.select('prediction','label')
#results.show(10)

In [ ]:
showMetrics(cross_results.rdd,value=1)
showMyMetric(results,v1 = 1,v2 = 0)